In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1649049,2021-09-12,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1649050,2021-09-13,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1649051,2021-09-14,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1649052,2021-09-15,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
43127,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
43129,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
43131,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
43133,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
43135,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1005880,2021-09-12,Arkansas,Arkansas,5001,3250,55.00,5000,Arkansas,AR,Arkansas,State,3017804
1005882,2021-09-13,Arkansas,Arkansas,5001,3255,55.00,5000,Arkansas,AR,Arkansas,State,3017804
1005884,2021-09-14,Arkansas,Arkansas,5001,3273,55.00,5000,Arkansas,AR,Arkansas,State,3017804
1005886,2021-09-15,Arkansas,Arkansas,5001,3294,55.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1649049,2021-09-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1649050,2021-09-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1649051,2021-09-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1649052,2021-09-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-09-16') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
604,2021-09-16,Snohomish,Washington,55543,673.00,53061,WA,County,822083,6756.37,81.87
1206,2021-09-16,Cook,Illinois,608383,11463.00,17031,IL,County,5150233,11812.73,222.57
1807,2021-09-16,Orange,California,313110,5337.00,6059,CA,County,3175692,9859.58,168.06
2407,2021-09-16,Maricopa,Arizona,671558,11110.00,4013,AZ,County,4485414,14972.04,247.69
3007,2021-09-16,Los Angeles,California,1439720,25775.00,6037,CA,County,10039107,14341.12,256.75
...,...,...,...,...,...,...,...,...,...,...,...
1647821,2021-09-16,Wheeler,Oregon,65,1.00,41069,OR,County,1332,4879.88,75.08
1648160,2021-09-16,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1648468,2021-09-16,Esmeralda,Nevada,46,0.00,32009,NV,County,873,5269.19,0.00
1648772,2021-09-16,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
604,2021-09-16,Snohomish,Washington,55543,673.00,53061,WA,County,822083,6756.37,81.87,81.87,6756.37
1206,2021-09-16,Cook,Illinois,608383,11463.00,17031,IL,County,5150233,11812.73,222.57,222.57,11812.73
1807,2021-09-16,Orange,California,313110,5337.00,6059,CA,County,3175692,9859.58,168.06,168.06,9859.58
2407,2021-09-16,Maricopa,Arizona,671558,11110.00,4013,AZ,County,4485414,14972.04,247.69,247.69,14972.04
3007,2021-09-16,Los Angeles,California,1439720,25775.00,6037,CA,County,10039107,14341.12,256.75,256.75,14341.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1647821,2021-09-16,Wheeler,Oregon,65,1.00,41069,OR,County,1332,4879.88,75.08,75.08,4879.88
1648160,2021-09-16,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1648468,2021-09-16,Esmeralda,Nevada,46,0.00,32009,NV,County,873,5269.19,0.00,0.00,5269.19
1648772,2021-09-16,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00,0.00,4733.73


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1052067,2021-09-16,Hale,Alabama,2840,81.00,1065,AL,County,14651,19384.34,552.86,552.86,19384.34,1
1145787,2021-09-16,Clarke,Alabama,4478,71.00,1025,AL,County,23622,18956.90,300.57,300.57,18956.90,2
728494,2021-09-16,Etowah,Alabama,18450,414.00,1055,AL,County,102268,18040.83,404.82,404.82,18040.83,3
610806,2021-09-16,Franklin,Alabama,5600,93.00,1059,AL,County,31362,17856.00,296.54,296.54,17856.00,4
729576,2021-09-16,Marshall,Alabama,17138,255.00,1095,AL,County,96774,17709.30,263.50,263.50,17709.30,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885261,2021-09-16,Johnson,Wyoming,1007,13.00,56019,WY,County,8445,11924.22,153.94,153.94,11924.22,19
1392370,2021-09-16,Niobrara,Wyoming,268,4.00,56027,WY,County,2356,11375.21,169.78,169.78,11375.21,20
1249724,2021-09-16,Lincoln,Wyoming,2139,14.00,56023,WY,County,19830,10786.69,70.60,70.60,10786.69,21
1004277,2021-09-16,Sublette,Wyoming,1032,11.00,56035,WY,County,9831,10497.41,111.89,111.89,10497.41,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
822081,2021-09-16,Lowndes,Alabama,1681,56.00,1085,AL,County,9726,17283.57,575.78,575.78,17283.57,10,1
1052067,2021-09-16,Hale,Alabama,2840,81.00,1065,AL,County,14651,19384.34,552.86,552.86,19384.34,1,2
886878,2021-09-16,Greene,Alabama,1182,41.00,1063,AL,County,8111,14572.80,505.49,505.49,14572.80,47,3
348381,2021-09-16,Walker,Alabama,10110,304.00,1127,AL,County,63521,15916.00,478.58,478.58,15916.00,31,4
727953,2021-09-16,Dallas,Alabama,4595,174.00,1047,AL,County,37196,12353.48,467.79,467.79,12353.48,61,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1218336,2021-09-16,Uinta,Wyoming,3303,21.00,56041,WY,County,20226,16330.47,103.83,103.83,16330.47,4,19
1567232,2021-09-16,Weston,Wyoming,848,7.00,56045,WY,County,6927,12241.95,101.05,101.05,12241.95,18,20
1249724,2021-09-16,Lincoln,Wyoming,2139,14.00,56023,WY,County,19830,10786.69,70.60,70.60,10786.69,21,21
818841,2021-09-16,Albany,Wyoming,5514,25.00,56001,WY,County,38880,14182.10,64.30,64.30,14182.10,10,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,81.87,6756.37,25,26
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,81.87,6756.37,25,26
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,81.87,6756.37,25,26
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,81.87,6756.37,25,26
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,81.87,6756.37,25,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1645220,2021-09-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1645221,2021-09-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1645222,2021-09-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1645223,2021-09-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
1047703,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19384.34,2,1,1.00
1047704,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19384.34,2,1,0.00
1047705,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19384.34,2,1,0.00
1047706,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19384.34,2,1,0.00
1047707,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,552.86,19384.34,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270848,2021-09-12,Crook,Wyoming,626,13.00,56011,WY,County,7584,8254.22,171.41,184.60,8887.13,9,23,0.00
1270849,2021-09-13,Crook,Wyoming,649,13.00,56011,WY,County,7584,8557.49,171.41,184.60,8887.13,9,23,23.00
1270850,2021-09-14,Crook,Wyoming,657,14.00,56011,WY,County,7584,8662.97,184.60,184.60,8887.13,9,23,8.00
1270851,2021-09-15,Crook,Wyoming,665,14.00,56011,WY,County,7584,8768.46,184.60,184.60,8887.13,9,23,8.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
818789,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,575.78,17283.57,1,10,1.00,0.00
818790,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,575.78,17283.57,1,10,0.00,0.00
818791,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,575.78,17283.57,1,10,0.00,0.00
818792,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,575.78,17283.57,1,10,0.00,0.00
818793,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,575.78,17283.57,1,10,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344524,2021-09-12,Teton,Wyoming,4611,11.00,56039,WY,County,23464,19651.38,46.88,46.88,20141.49,23,1,0.00,0.00
344525,2021-09-13,Teton,Wyoming,4655,11.00,56039,WY,County,23464,19838.90,46.88,46.88,20141.49,23,1,44.00,0.00
344526,2021-09-14,Teton,Wyoming,4679,11.00,56039,WY,County,23464,19941.19,46.88,46.88,20141.49,23,1,24.00,0.00
344527,2021-09-15,Teton,Wyoming,4694,11.00,56039,WY,County,23464,20005.11,46.88,46.88,20141.49,23,1,15.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-09-16') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
416177,2021-09-16,Imperial,California,34030,752.00,6025,CA,County,181215,18778.80,414.98,414.98,18778.80,1,3,138.00,1.00
3007,2021-09-16,Los Angeles,California,1439720,25775.00,6037,CA,County,10039107,14341.12,256.75,256.75,14341.12,2,9,1938.00,27.00
206681,2021-09-16,San Bernardino,California,345071,5470.00,6071,CA,County,2180085,15828.33,250.91,250.91,15828.33,3,4,749.00,5.00
92157,2021-09-16,Stanislaus,California,79718,1222.00,6099,CA,County,550660,14476.81,221.92,221.92,14476.81,4,7,298.00,12.00
730610,2021-09-16,Inyo,California,1640,39.00,6027,CA,County,18039,9091.41,216.20,216.20,9091.41,5,36,26.00,0.00
73255,2021-09-16,San Joaquin,California,96242,1619.00,6077,CA,County,762148,12627.73,212.43,212.43,12627.73,6,16,381.00,0.00
42567,2021-09-16,Riverside,California,355499,4890.00,6065,CA,County,2470546,14389.49,197.93,197.93,14389.49,7,8,686.00,18.00
661793,2021-09-16,Merced,California,39121,538.00,6047,CA,County,277680,14088.52,193.75,193.75,14088.52,8,10,138.00,3.00
114354,2021-09-16,Tulare,California,71958,890.00,6107,CA,County,466195,15435.17,190.91,190.91,15435.17,9,5,838.00,2.00
41449,2021-09-16,Fresno,California,134246,1867.00,6019,CA,County,999101,13436.68,186.87,186.87,13436.68,10,13,473.00,11.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1556206,2021-09-16,Lassen,California,6785,32.00,6035,CA,County,30573,22192.78,104.67,104.67,22192.78,33,1,20.00,0.00
886820,2021-09-16,Kings,California,29699,279.00,6031,CA,County,152940,19418.73,182.42,182.42,19418.73,11,2,139.00,1.00
416177,2021-09-16,Imperial,California,34030,752.00,6025,CA,County,181215,18778.80,414.98,414.98,18778.80,1,3,138.00,1.00
206681,2021-09-16,San Bernardino,California,345071,5470.00,6071,CA,County,2180085,15828.33,250.91,250.91,15828.33,3,4,749.00,5.00
114354,2021-09-16,Tulare,California,71958,890.00,6107,CA,County,466195,15435.17,190.91,190.91,15435.17,9,5,838.00,2.00
260073,2021-09-16,Kern,California,133792,1514.00,6029,CA,County,900202,14862.44,168.18,168.18,14862.44,13,6,510.00,4.00
92157,2021-09-16,Stanislaus,California,79718,1222.00,6099,CA,County,550660,14476.81,221.92,221.92,14476.81,4,7,298.00,12.00
42567,2021-09-16,Riverside,California,355499,4890.00,6065,CA,County,2470546,14389.49,197.93,197.93,14389.49,7,8,686.00,18.00
3007,2021-09-16,Los Angeles,California,1439720,25775.00,6037,CA,County,10039107,14341.12,256.75,256.75,14341.12,2,9,1938.00,27.00
661793,2021-09-16,Merced,California,39121,538.00,6047,CA,County,277680,14088.52,193.75,193.75,14088.52,8,10,138.00,3.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
416177,2021-09-16,Imperial,California,34030,752.00,6025,CA,County,181215,18778.80,414.98,414.98,18778.80,1,3,138.00,1.00
3007,2021-09-16,Los Angeles,California,1439720,25775.00,6037,CA,County,10039107,14341.12,256.75,256.75,14341.12,2,9,1938.00,27.00
206681,2021-09-16,San Bernardino,California,345071,5470.00,6071,CA,County,2180085,15828.33,250.91,250.91,15828.33,3,4,749.00,5.00
92157,2021-09-16,Stanislaus,California,79718,1222.00,6099,CA,County,550660,14476.81,221.92,221.92,14476.81,4,7,298.00,12.00
730610,2021-09-16,Inyo,California,1640,39.00,6027,CA,County,18039,9091.41,216.20,216.20,9091.41,5,36,26.00,0.00
73255,2021-09-16,San Joaquin,California,96242,1619.00,6077,CA,County,762148,12627.73,212.43,212.43,12627.73,6,16,381.00,0.00
42567,2021-09-16,Riverside,California,355499,4890.00,6065,CA,County,2470546,14389.49,197.93,197.93,14389.49,7,8,686.00,18.00
661793,2021-09-16,Merced,California,39121,538.00,6047,CA,County,277680,14088.52,193.75,193.75,14088.52,8,10,138.00,3.00
114354,2021-09-16,Tulare,California,71958,890.00,6107,CA,County,466195,15435.17,190.91,190.91,15435.17,9,5,838.00,2.00
41449,2021-09-16,Fresno,California,134246,1867.00,6019,CA,County,999101,13436.68,186.87,186.87,13436.68,10,13,473.00,11.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,3,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,3,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5472,09/12/21,Lassen,6577,31.00,21512.45,101.40,33,1,0.00,0.00
5473,09/13/21,Lassen,6603,31.00,21597.49,101.40,33,1,26.00,0.00
5474,09/14/21,Lassen,6741,32.00,22048.87,104.67,33,1,138.00,1.00
5475,09/15/21,Lassen,6765,32.00,22127.37,104.67,33,1,24.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,3,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,3,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5558,09/12/21,Fresno,132846,1853.00,13296.55,185.47,10,13,565.00,7.00
5559,09/13/21,Fresno,133029,1853.00,13314.87,185.47,10,13,183.00,0.00
5560,09/14/21,Fresno,133272,1853.00,13339.19,185.47,10,13,243.00,0.00
5561,09/15/21,Fresno,133773,1856.00,13389.34,185.77,10,13,501.00,3.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)